# The TinyALU TLM Testbench

In [1]:
#!/usr/bin/env python
# coding: utf-8

# # The TinyALU TLM Testbench

# In[1]:

from tb_pkg import *
from pyuvm import *


            

class Driver(uvm_component):
    def build_phase(self):
        self.proxy = self.cdb_get("PROXY")
    
    def run_phase(self):
        self.raise_objection()
        for _ in range(5):
            aa = random.randrange(256)
            bb = random.randrange(256)
            op = random.choice(list(Ops))
            self.proxy.send_op(aa, bb, op)
        time.sleep(0.6) # Wait for last operation to complete
        self.drop_objection()   


class Coverage(uvm_subscriber):
    
    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def write(self, cmd):
        (_, _, op) = cmd
        self.cvg.add(op)

    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")


class Scoreboard(uvm_component):  

    def build_phase(self):
        self.cmd_fifo = uvm_tlm_analysis_fifo("cmd_fifo", self)
        self.result_fifo = uvm_tlm_analysis_fifo("result_fifo", self)
        self.cmd_get_port = uvm_get_port("cmd_get_port", self)    
        self.result_get_port = uvm_get_port("result_get_port", self)
        self.cmd_export = self.cmd_fifo.analysis_export
        self.result_export = self.result_fifo.analysis_export

    def connect_phase(self):
        self.cmd_get_port.connect(self.cmd_fifo.get_export)
        self.result_get_port.connect(self.result_fifo.get_export)

    def check_phase(self):
        while self.result_get_port.can_get():
            _, actual_result = self.result_get_port.try_get()
            cmd_success, (A, B, op_numb) = self.cmd_get_port.try_get()
            if not cmd_success:
                self.logger.critical(f"result {actual_result} had no command")
            else:
                op = Ops(op_numb)
                predicted_result = PythonProxy.alu_op(A, B, op)
                if predicted_result == actual_result:
                    self.logger.info(f"PASSED: 0x{A:02x} {op.name} 0x{B:02x} ="
                                     f" 0x{actual_result:04x}")
                else:
                    self.logger.error(f"FAILED: 0x{A:02x} {op.name} 0x{B:02x} "
                                      f"= 0x{actual_result:04x} expected 0x{predicted_result:04x}")


class Monitor(uvm_component):
    def __init__(self, name, parent, method_name):
        super().__init__(name, parent)
        self.method_name = method_name
    
    def build_phase(self):
        self.proxy = self.cdb_get("PROXY")
        self.ap = uvm_analysis_port("ap", self)

    def run_phase(self):
        while not ObjectionHandler().run_phase_complete():
            get_method = getattr(self.proxy, self.method_name)
            datum = get_method()
            self.ap.write(datum)    


class AluEnv(uvm_env):

    def build_phase(self):
        self.cmd_mon = Monitor("cmd_mon", self, "get_cmd")
        self.result_mon = Monitor("result_mon", self, "get_result")
        self.scoreboard = Scoreboard("scoreboard", self)
        self.coverage = Coverage("coverage", self)
        self.driver = Driver("driver", self)
        ConfigDB().set(None, "*", "CVG", self.coverage)
        
    def connect_phase(self):
        self.cmd_mon.ap.connect(self.scoreboard.cmd_export)
        self.cmd_mon.ap.connect(self.coverage)
        self.result_mon.ap.connect(self.scoreboard.result_export)


class AluTest(uvm_test):
    def build_phase(self):
        model_proxy = PythonProxy("model_proxy", self)
        ConfigDB().set(None, "*", "PROXY", model_proxy)
        self.env = AluEnv("env", self)

if __name__ == "__main__":
    uvm_root().run_test("AluTest")


INFO: <ipython-input-1-59464429a852>(67)[uvm_test_top.env.scoreboard]: PASSED: 0xc7 MUL 0x84 = 0x669c
INFO: <ipython-input-1-59464429a852>(67)[uvm_test_top.env.scoreboard]: PASSED: 0x98 XOR 0x9c = 0x0004
INFO: <ipython-input-1-59464429a852>(67)[uvm_test_top.env.scoreboard]: PASSED: 0x35 AND 0xb1 = 0x0031
INFO: <ipython-input-1-59464429a852>(67)[uvm_test_top.env.scoreboard]: PASSED: 0x49 AND 0xf3 = 0x0041
INFO: <ipython-input-1-59464429a852>(67)[uvm_test_top.env.scoreboard]: PASSED: 0xbd AND 0xfd = 0x00bd
ERROR: <ipython-input-1-59464429a852>(40)[uvm_test_top.env.coverage]: Functional coverage error. Missed: {<Ops.ADD: 1>}
